In [1]:
include("../src/juisee/Backend.jl")
using .Backend

### The Backend.jl module
is an all-Julia implementation of a simple vector database. The general idea is that we need a way to do the following:
1. Store textual metadata, such as document names, section/chunk text, etc.
2. Store vector embeddings
3. Search vector embeddings
4. Retrieval textual metadata from our vector search
This ends up being a bit convoluted because the system best for 2 and 3 does not necessarily support 1 and 4.

We resolve this by creating a Corpus struct, which combines a fast vector index (Hierarchical Navigable Small World) with a traditional relational database (SQLite). The former handles items 2 and 3 and the latter handles 1 and 4. The details are abstracted away via the Corpus' upsert and search methods.

Instantiating a Corpus in-memory (e.g. don't save the associated database to disk) is straightforward (saving to disk requires passing a corpus_name argument):

In [2]:
corpus = Corpus()

Corpus(SQLite.DB(":memory:"), nothing, Main.Backend.Embedding.Embedder(BertTextEncoder(
├─ TextTokenizer(MatchTokenization(WordPieceTokenization(bert_uncased_tokenizer, WordPiece(vocab_size = 30522, unk = [UNK], max_char = 100)), 5 patterns)),
├─ vocab = Vocab{String, SizedArray}(size = 30522, unk = [UNK], unki = 101),
├─ startsym = [CLS],
├─ endsym = [SEP],
├─ padsym = [PAD],
├─ trunc = 512,
└─ process = Pipelines:
  ╰─ target[token] := TextEncodeBase.nestedcall(string_getvalue, source)
  ╰─ target[token] := Transformers.TextEncoders.grouping_sentence(target.token)
  ╰─ target[(token, segment)] := SequenceTemplate{String}([CLS]:<type=1> Input[1]:<type=1> [SEP]:<type=1> (Input[2]:<type=2> [SEP]:<type=2>)...)(target.token)
  ╰─ target[attention_mask] := (NeuralAttentionlib.LengthMask ∘ Transformers.TextEncoders.getlengths(512))(target.token)
  ╰─ target[token] := TextEncodeBase.trunc_and_pad(512, [PAD], tail, tail)(target.token)
  ╰─ target[token] := TextEncodeBase.nested2batch(target.t

We can upsert small chunks of text with associated document names:

In [3]:
upsert_chunk(corpus, "Hold me closer, tiny dancer.", "doc1")
upsert_chunk(corpus, "Count the headlights on the highway.", "doc1")
upsert_chunk(corpus, "Lay me down in sheets of linen.", "doc2")
upsert_chunk(corpus, "Peter Piper picked a peck of pickled peppers. A peck of pickled peppers, Peter Piper picked.", "doc2")

5

Then we can search our vector index to get the chunks closest to our query:

In [4]:
idx_list, doc_names, chunks, distances = Backend.search(
    corpus, 
    "tiny dancer", 
    2
)

([1, 3], ["doc1", "doc2"], ["Hold me closer, tiny dancer.", "Lay me down in sheets of linen."], Vector{Float32}[[5.198073, 9.5337925]])

As expected, the chunk closest to "tiny dancer" is the one that says "Hold me closer, tiny dancer." Nice!

We can also upsert whole-documents and automate the chunking. Let's try that with George Washington's 1789 Inaugural Address, which is large enough to warrant several chunks:

In [5]:
washington_inaugural_address = """
Fellow Citizens of the Senate and the House of Representatives.
Among the vicissitudes incident to life, no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the fourteenth day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years: a retreat which was rendered every day more necessary as well as more dear to me, by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my Country called me, being sufficient to awaken in the wisest and most experienced of her citizens, a distrustful scrutiny into his qualifications, could not but overwhelm with dispondence, one, who, inheriting inferior endowments from nature and unpractised in the duties of civil administration, ought to be peculiarly conscious of his own deficiencies. In this conflict of emotions, all I dare aver, is, that it has been my faithful study to collect my duty from a just appreciation of eve ry circumstance, by which it might be affected. All I dare hope, is, that, if in executing this task I have been too much swayed by a grateful remembrance of former instances, or by an affectionate sensibility to this transcendent proof, of the confidence of my fellow-citizens; and have thence too little consulted my incapacity as well as disinclination for the weighty and untried cares before me; my error will be palliated by the motives which misled me, and its consequences be judged by my Country, with some share of the partiality in which they originated.
Such being the impressions under which I have, in obedience to the public summons, repaired to the present station; it would be peculiarly improper to omit in this first official Act, my fervent supplications to that Almighty Being who rules over the Universe, who presides in the Councils of Nations, and whose providential aids can supply every human defect, that his benediction may consecrate to the liberties and happiness of the People of the United States, a Government instituted by themselves for these essential purposes: and may enable every instrument employed in its administration to execute with success, the functions allotted to his charge. In tendering this homage to the Great Author of every public and private good I assure myself that it expresses your sentiments not less than my own; nor those of my fellow-citizens at large, less than either. No People can be bound to acknowledge and adore the invisible hand, which conducts the Affairs of men more than the People of the United States. Every step, by which they have advanced to the character of an independent nation, seems to have been distinguished by some token of providential agency. And in the important revolution just accomplished in the system of their United Government, the tranquil deliberations and voluntary consent of so many distinct communities, from which the event has resulted, cannot be compared with the means by which most Governments have been established, without some return of pious gratitude along with an humble anticipation of the future blessings which the past seem to presage. These reflections, arising out of the present crisis, have forced themselves too strongly on my mind to be suppressed. You will join with me I trust in thinking, that there are none under the influence of which, the proceedings of a new and free Government can more auspiciously commence.
By the article establishing the Executive Department, it is made the duty of the President "to recommend to your consideration, such measures as he shall judge necessary and expedient." The circumstances under which I now meet you, will acquit me from entering into that subject, farther than to refer to the Great Constitutional Charter under which you are assembled; and which, in defining your powers, designates the objects to which your attention is to be given. It will be more consistent with those circumstances, and far more congenial with the feelings which actuate me, to substitute, in place of a recommendation of particular measures, the tribute that is due to the talents, the rectitude, and the patriotism which adorn the characters selected to devise and adopt them. In these honorable qualifications, I behold the surest pledges, that as on one side, no local prejudices, or attachments; no seperate views, nor party animosities, will misdirect the comprehensive and equal eye which ought to watch over this great assemblage of communities and interests: so, on another, that the foundations of our National policy will be laid in the pure and immutable principles of private morality; and the pre-eminence of a free Government, be exemplified by all the attributes which can win the affections of its Citizens, and command the respect of the world.
I dwell on this prospect with every satisfaction which an ardent love for my Country can inspire: since there is no truth more thoroughly established, than that there exists in the economy and course of nature, an indissoluble union between virtue and happiness, between duty and advantage, between the genuine maxims of an honest and magnanimous policy, and the solid rewards of public prosperity and felicity: Since we ought to be no less persuaded that the propitious smiles of Heaven, can never be expected on a nation that disregards the eternal rules of order and right, which Heaven itself has ordained: And since the preservation of the sacred fire of liberty, and the destiny of the Republican model of Government, are justly considered as deeply, perhaps as finally staked, on the experiment entrusted to the hands of the American people.
Besides the ordinary objects submitted to your care, it will remain with your judgment to decide, how far an exercise of the occasional power delegated by the Fifth article of the Constitution is rendered expedient at the present juncture by the nature of objections which have been urged against the System, or by the degree of inquietude which has given birth to them. Instead of undertaking particular recommendations on this subject, in which I could be guided by no lights derived from official opportunities, I shall again give way to my entire confidence in your discernment and pursuit of the public good: For I assure myself that whilst you carefully avoid every alteration which might endanger the benefits of an United and effective Government, or which ought to await the future lessons of experience; a reverence for the characteristic rights of freemen, and a regard for the public harmony, will sufficiently influence your deliberations on the question how far the former can be more impregnably fortified, or the latter be safely and advantageously promoted.
To the preceeding observations I have one to add, which will be most properly addressed to the House of Representatives. It concerns myself, and will therefore be as brief as possible. When I was first honoured with a call into the Service of my Country, then on the eve of an arduous struggle for its liberties, the light in which I contemplated my duty required that I should renounce every pecuniary compensation. From this resolution I have in no instance departed. And being still under the impressions which produced it, I must decline as inapplicable to myself, any share in the personal emoluments, which may be indispensably included in a permanent provision for the Executive Department; and must accordingly pray that the pecuniary estimates for the Station in which I am placed, may, during my continuance in it, be limited to such actual expenditures as the public good may be thought to require.
Having thus imparted to you my sentiments, as they have been awakened by the occasion which brings us together, I shall take my present leave; but not without resorting once more to the benign parent of the human race, in humble supplication that since he has been pleased to favour the American people, with opportunities for deliberating in perfect tranquility, and dispositions for deciding with unparellelled unanimity on a form of Government, for the security of their Union, and the advancement of their happiness; so his divine blessing may be equally conspicuous in the enlarged views, the temperate consultations, and the wise measures on which the success of this Government must depend.
"""

"Fellow Citizens of the Senate and the House of Representatives.\nAmong the vicissitudes incident to life, no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the fourteenth day of the pre" ⋯ 8139 bytes ⋯ "form of Government, for the security of their Union, and the advancement of their happiness; so his divine blessing may be equally conspicuous in the enlarged views, the temperate consultations, and the wise measures on which the success of this Government must depend.\n"

In [6]:
upsert_document(corpus, washington_inaugural_address, "George Washington Inaugural Address 1789")

Reasonably, the top results from searching "being called to serve one's country" are all subsections of the inaugural address:

In [7]:
idx_list, doc_names, chunks, distances = Backend.search(
    corpus, 
    "being called to serve one's country", 
    3
)

([5, 7, 6], ["George Washington Inaugural Address 1789", "George Washington Inaugural Address 1789", "George Washington Inaugural Address 1789"], ["Fellow Citizens of the Senate and the House of Representatives. Among the vicissitudes incident to life, no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the fourteenth day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years: a retreat which was rendered every day more necessary as well as more dear to me, by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my Country